In [1]:
import os
import pandas as pd
import numpy as np
os.chdir(r"C:\Users\sd4be\Downloads\credit_data")

In [2]:
df=pd.read_csv("funda_train.csv")

### 1 Extract Features 

In [3]:
import copy
def extract_features(date):
    date=str(date).split('-')
    return int(date[0]),int(date[1])

#extract year and month from timestamp date
df['year'],df['month']=zip(*df.transacted_date.map(lambda x:extract_features(x)))    

In [4]:
# create a dummy variable t
df['t']=(df['year']-2016)*12+df['month']



#subjective opinion - drop unrelated columns for the prediction
df.drop(['year','month','transacted_date','transacted_time'],axis=1,inplace=True)
df.drop(['card_id','card_company'],axis=1,inplace=True)

### 2. General Overview of Data

### 2.1 Installment

In [5]:
%%time
installment=df.installment_term.value_counts()/df.installment_term.value_counts().sum()
installment=(installment.sort_index().round(2)*100).astype(str)+'%'

Wall time: 76 ms


In [6]:
installment=pd.DataFrame(installment)
installment['counts']=df.installment_term.value_counts()
#prent the first 10 highet counts 
installment[:10]

,installment_term,counts
0,97.0%,6327632
2,1.0%,42101
3,2.0%,134709
4,0.0%,4816
5,0.0%,23751
6,0.0%,10792
7,0.0%,553
8,0.0%,413
9,0.0%,349
10,0.0%,6241


In [7]:
#categorize the installment_term
df['is_installment']=(df['installment_term']>0).astype(int)
df.head()

,store_id,installment_term,region,type_of_business,amount,t,is_installment
0,0,0,NaN,기타 미용업,1857.142857,6,0
1,0,0,NaN,기타 미용업,857.142857,6,0
2,0,0,NaN,기타 미용업,2000.000000,6,0
3,0,0,NaN,기타 미용업,7857.142857,6,0
4,0,0,NaN,기타 미용업,2000.000000,6,0


In [8]:
# proportion of instalment 
pr_installment=df.groupby("store_id")['is_installment'].mean().sort_values(ascending=False)
pr_installment

store_id
57      0.904215
1629    0.767231
398     0.755750
1626    0.720287
1699    0.717949
          ...   
1247    0.000000
478     0.000000
479     0.000000
50      0.000000
1927    0.000000
Name: is_installment, Length: 1967, dtype: float64

### 2.2 filling Values

In [9]:
df['region'].fillna("None",inplace=True)
df['type_of_business'].fillna("None",inplace=True)


In [10]:
# 'store_id','region','type_of_business','t' 
train_df=df.drop_duplicates(subset=['store_id','region','type_of_business','t'])[['store_id','region','type_of_business','t']]
train_df.head()

,store_id,region,type_of_business,t
0,0,None,기타 미용업,6
145,0,None,기타 미용업,7
323,0,None,기타 미용업,8
494,0,None,기타 미용업,9
654,0,None,기타 미용업,10


### 3.Feature Engineering

### 3.1 Total Monthly Amount

In [11]:
#add the mean proportion of installment to the train data
train_df['installment_prop_mean']=train_df.store_id.replace(pr_installment.to_dict())

In [12]:
# attach t-1,t-2,t-3
total_amount_t_sid=df.groupby(['store_id','t'],as_index=False)['amount'].sum()
total_amount_t_sid.head()

,store_id,t,amount
0,0,6,7.470000e+05
1,0,7,1.005000e+06
2,0,8,8.715714e+05
3,0,9,8.978571e+05
4,0,10,8.354286e+05


Some stores have an equal number of t's,which implies that we can observe missing data available.

In [13]:
total_amount_t_sid.groupby("store_id")['t'].count().head(10)

store_id
0     33
1     33
2     33
4     33
5     33
6     31
7     31
8     28
9     29
10    23
Name: t, dtype: int64

### 3.1.1 Filling Missing Values

As explained earlier,we observe some stores have missing records on the specific months. The missing values can be clearly 
seen when we present the infomration in the format of pivot-table. 

In [14]:
%%time
amount_sum=pd.pivot_table(total_amount_t_sid,index='store_id',columns="t",aggfunc="sum")

amount_sum.head(10)

Wall time: 40.9 ms


amount                                             \
t                    6             7              8             9    
store_id                                                             
0         747000.000000  1.005000e+06  871571.428571  8.978571e+05   
1         137214.285714  1.630000e+05  118142.857143  9.042857e+04   
2         260714.285714  8.285714e+04  131428.571429  1.428571e+05   
4         733428.571429  7.689286e+05  698428.571429  9.364286e+05   
5         342500.000000  4.327143e+05  263500.000000  2.321429e+05   
6                   NaN           NaN  568857.142857  1.440143e+06   
7                   NaN           NaN  107857.142857  3.756429e+05   
8                   NaN           NaN            NaN           NaN   
9                   NaN           NaN            NaN           NaN   
10                  NaN           NaN            NaN           NaN   

                                                                   \
t                   10            11            12             13   
store_id                                                            
0         8.354286e+05  6.970000e+05  7.618571e+05  585642.857143   
1         1.180714e+05  1.118571e+05  1.155714e+05  129642.857143   
2         1.097143e+05  1.985714e+05  1.600000e+05  180714.285714   
4         7.627143e+05  8.595714e+05  1.069857e+06  689142.857143   
5         2.115714e+05  1.820857e+05  1.475714e+05  120957.142857   
6         1.238857e+06  1.055429e+06  9.268571e+05  885642.857143   
7         3.236429e+05  3.450000e+05  2.914286e+05  231614.285714   
8                  NaN  1.925714e+05  7.355000e+05  467857.142857   
9         1.071429e+05  6.371429e+05  6.035714e+05  225428.571429   
10                 NaN           NaN           NaN            NaN   

                                       ...                              \
t                   14             15  ...            29            30   
store_id                               ...                               
0         7.940000e+05  720257.142857  ...  6.864286e+05  7.072857e+05   
1         1.602143e+05  168428.571429  ...  8.050000e+04  7.828571e+04   
2         1.542857e+05   43571.428571  ...  4.728571e+05  3.542857e+05   
4         1.050143e+06  970285.714286  ...  7.754286e+05  8.812857e+05   
5         1.864286e+05  169000.000000  ...  4.438571e+05  5.637143e+05   
6         8.003571e+05  930714.285714  ...  1.808357e+06  1.752286e+06   
7         2.713571e+05  249857.142857  ...  2.657143e+05  4.195429e+05   
8         4.756429e+05  603500.000000  ...  1.837429e+06  1.359857e+06   
9         2.871429e+05  344428.571429  ...  6.385714e+05  2.765714e+05   
10                 NaN            NaN  ...  2.902857e+05  6.078571e+05   

                                                                  \
t                   31            32            33            34   
store_id                                                           
0         7.587143e+05  6.798571e+05  6.518571e+05  7.390000e+05   
1         1.007857e+05  9.214286e+04  6.357143e+04  9.500000e+04   
2         6.892857e+05  4.578571e+05  4.807143e+05  5.100000e+05   
4         1.050929e+06  8.492857e+05  6.981429e+05  8.284286e+05   
5         6.070714e+05  4.828857e+05  1.950000e+05  3.249286e+05   
6         1.583786e+06  1.628786e+06  2.074071e+06  1.907643e+06   
7         4.628429e+05  4.231286e+05  3.203286e+05  4.200286e+05   
8         1.213543e+06  1.086000e+06  1.369557e+06  1.272071e+06   
9         3.400000e+05  2.542857e+05  9.265714e+05  8.714286e+05   
10        4.445714e+05  6.414286e+05  7.955714e+05  4.992857e+05   

                                                                  
t                   35            36            37            38  
store_id                                                          
0         6.760000e+05  8.745714e+05  6.828571e+05  5.152857e+05  
1         8.078571e+04  8.528571e+04  1.482857e+05  7.742857e+04  
2         1.854286e+05  3

Before taking special treatment to the missing values,we need to make an assumption. 

"All the missing values are closely related to those in recent period." Based on this perception,we will fill all the values by the 'bfill' and 'ffill'.



In [15]:
amount_sum=amount_sum.fillna(method='ffill',axis=1).fillna(method='bfill',axis=1)
amount_sum.head(10)

amount                                             \
t                    6             7              8             9    
store_id                                                             
0         747000.000000  1.005000e+06  871571.428571  8.978571e+05   
1         137214.285714  1.630000e+05  118142.857143  9.042857e+04   
2         260714.285714  8.285714e+04  131428.571429  1.428571e+05   
4         733428.571429  7.689286e+05  698428.571429  9.364286e+05   
5         342500.000000  4.327143e+05  263500.000000  2.321429e+05   
6         568857.142857  5.688571e+05  568857.142857  1.440143e+06   
7         107857.142857  1.078571e+05  107857.142857  3.756429e+05   
8         192571.428571  1.925714e+05  192571.428571  1.925714e+05   
9         107142.857143  1.071429e+05  107142.857143  1.071429e+05   
10        496714.285714  4.967143e+05  496714.285714  4.967143e+05   

                                                                   \
t                   10            11            12             13   
store_id                                                            
0         8.354286e+05  6.970000e+05  7.618571e+05  585642.857143   
1         1.180714e+05  1.118571e+05  1.155714e+05  129642.857143   
2         1.097143e+05  1.985714e+05  1.600000e+05  180714.285714   
4         7.627143e+05  8.595714e+05  1.069857e+06  689142.857143   
5         2.115714e+05  1.820857e+05  1.475714e+05  120957.142857   
6         1.238857e+06  1.055429e+06  9.268571e+05  885642.857143   
7         3.236429e+05  3.450000e+05  2.914286e+05  231614.285714   
8         1.925714e+05  1.925714e+05  7.355000e+05  467857.142857   
9         1.071429e+05  6.371429e+05  6.035714e+05  225428.571429   
10        4.967143e+05  4.967143e+05  4.967143e+05  496714.285714   

                                       ...                              \
t                   14             15  ...            29            30   
store_id                               ...                               
0         7.940000e+05  720257.142857  ...  6.864286e+05  7.072857e+05   
1         1.602143e+05  168428.571429  ...  8.050000e+04  7.828571e+04   
2         1.542857e+05   43571.428571  ...  4.728571e+05  3.542857e+05   
4         1.050143e+06  970285.714286  ...  7.754286e+05  8.812857e+05   
5         1.864286e+05  169000.000000  ...  4.438571e+05  5.637143e+05   
6         8.003571e+05  930714.285714  ...  1.808357e+06  1.752286e+06   
7         2.713571e+05  249857.142857  ...  2.657143e+05  4.195429e+05   
8         4.756429e+05  603500.000000  ...  1.837429e+06  1.359857e+06   
9         2.871429e+05  344428.571429  ...  6.385714e+05  2.765714e+05   
10        4.967143e+05  496714.285714  ...  2.902857e+05  6.078571e+05   

                                                                  \
t                   31            32            33            34   
store_id                                                           
0         7.587143e+05  6.798571e+05  6.518571e+05  7.390000e+05   
1         1.007857e+05  9.214286e+04  6.357143e+04  9.500000e+04   
2         6.892857e+05  4.578571e+05  4.807143e+05  5.100000e+05   
4         1.050929e+06  8.492857e+05  6.981429e+05  8.284286e+05   
5         6.070714e+05  4.828857e+05  1.950000e+05  3.249286e+05   
6         1.583786e+06  1.628786e+06  2.074071e+06  1.907643e+06   
7         4.628429e+05  4.231286e+05  3.203286e+05  4.200286e+05   
8         1.213543e+06  1.086000e+06  1.369557e+06  1.272071e+06   
9         3.400000e+05  2.542857e+05  9.265714e+05  8.714286e+05   
10        4.445714e+05  6.414286e+05  7.955714e+05  4.992857e+05   

                                                                  
t                   35            36            37            38  
store_id                                                          
0         6.760000e+05  8.745714e+05  6.828571e+05  5.152857e+05  
1         8.078571e+04  8.528571e+04  1.482857e+05  7.742857e+04  
2         1.854286e+05  3

In [16]:
#convert the data into the original format
amount_sum=amount_sum.stack().reset_index()
regions=df.groupby(["region","t"])["amount"].mean().reset_index()


### 3.2  Create Adttional Features : Total Amount for Previous Months

As the headline says, we will create the number of features indicating the total amount for previous mohths. The number is 
entirely up to your choice but in this case we will consider the past 3 months values. 


In [17]:
import sys
#attacth the amounts for the n number of months
label,ref_id="mean_previous",'store_id'
n=3
for k in range(1,n+1):
    t_label,value_label="t_{}".format(k),"{}_{}".format(label,k)
    amount_sum[t_label]=amount_sum['t']+k
    df=pd.merge(df,amount_sum.drop(columns='t').rename(columns={'amount':value_label}),left_on=[ref_id,'t'],right_on=[ref_id,t_label])
    df.drop(columns=[t_label],inplace=True)
    amount_sum.drop(columns=t_label,inplace=True)
        


### 3.3  Add New Features for Histrocial Average Values according to Regions

In [18]:
label,ref_id='region_previous','region'
for k in range(1,n+1):
    t_label,value_label="t_{}".format(k),"{}_{}".format(label,k)
    regions[t_label]=regions['t']+k
    df=pd.merge(df,regions.drop(columns='t').rename(columns={'amount':value_label}),left_on=[ref_id,'t'],right_on=[ref_id,t_label])
    df.drop(columns=[t_label],inplace=True)
    regions.drop(columns=t_label,inplace=True)

In [19]:
df.head()

,store_id,installment_term,region,type_of_business,amount,t,is_installment,mean_previous_1,mean_previous_2,mean_previous_3,region_previous_1,region_previous_2,region_previous_3
0,0,0,None,기타 미용업,2000.000000,9,0,871571.428571,1005000.0,747000.0,10907.070352,11031.902535,10895.011409
1,0,0,None,기타 미용업,2000.000000,9,0,871571.428571,1005000.0,747000.0,10907.070352,11031.902535,10895.011409
2,0,0,None,기타 미용업,6428.571429,9,0,871571.428571,1005000.0,747000.0,10907.070352,11031.902535,10895.011409
3,0,0,None,기타 미용업,7142.857143,9,0,871571.428571,1005000.0,747000.0,10907.070352,11031.902535,10895.011409
4,0,0,None,기타 미용업,1857.142857,9,0,871571.428571,1005000.0,747000.0,10907.070352,11031.902535,10895.011409


## 3.4. Add Mean Values by business type

In [20]:
t_amoun=amount_sum.copy()
temp=df.copy()

In [21]:
df=temp.copy()

In [22]:
amount_sum['type_of_business']=amount_sum.store_id.replace(df[['type_of_business',"store_id"]].drop_duplicates().set_index("store_id")['type_of_business'].to_dict())

In [23]:
amount_sum.head()

,store_id,t,amount,type_of_business
0,0,6,7.470000e+05,기타 미용업
1,0,7,1.005000e+06,기타 미용업
2,0,8,8.715714e+05,기타 미용업
3,0,9,8.978571e+05,기타 미용업
4,0,10,8.354286e+05,기타 미용업


In [31]:
business_mean=amount_sum.groupby(["type_of_business","t"],as_index=False)["amount"].mean()

In [32]:
label="business_previous"
for k in range(1,4):
    t_label,value_label='t_{}'.format(k),'{}-{}'.format(label,k)
    business_mean[t_label]=business_mean['t']+k
    df=pd.merge(df,business_mean.drop(columns='t').rename(columns={'amount':value_label}),\
               left_on=['type_of_business','t'],right_on=['type_of_business',t_label])
    df.drop(columns=t_label,inplace=True)
    business_mean.drop(columns=t_label,inplace=True)

In [33]:
df.head()
amount_sum.head()

,store_id,t,amount,type_of_business
0,0,6,7.470000e+05,기타 미용업
1,0,7,1.005000e+06,기타 미용업
2,0,8,8.715714e+05,기타 미용업
3,0,9,8.978571e+05,기타 미용업
4,0,10,8.354286e+05,기타 미용업


In [34]:
del business_mean
amount_sum.drop(columns='type_of_business',inplace=True)

### 3.4 Add Columns for the Next Three months Values

The competition rule states that the target variable should be the sum of the preivous 3 months total amount. 

To meet the requirement,we need to compute the amount for next 3 months from the reference month which is indicated by 't' column in our dataset.And based on the calculated sections, we are now able to caculate target variable,y by suming all the three.

In [35]:
label,ref_id='value_next','store_id'
for k in range(1,n+1):
    t_label,value_label="t_{}".format(k),"{}_{}".format(label,k)
    amount_sum[t_label]=amount_sum['t']-k
    df=pd.merge(df,amount_sum.drop(columns='t').rename(columns={'amount':value_label}),left_on=[ref_id,'t'],right_on=[ref_id,t_label])
    df.drop(columns=[t_label],inplace=True)
    amount_sum.drop(columns=t_label,inplace=True)

In [36]:
df['y']=df['value_next_1']+df['value_next_2']+df['value_next_3']

In [37]:
df.head()

,store_id,installment_term,region,type_of_business,amount,t,is_installment,mean_previous_1,mean_previous_2,mean_previous_3,region_previous_1,region_previous_2,region_previous_3,business_previous-1,business_previous-2,business_previous-3,value_next_1,value_next_2,value_next_3,y
0,0,0,None,기타 미용업,2000.000000,9,0,871571.428571,1005000.0,747000.0,10907.070352,11031.902535,10895.011409,761025.0,804979.761905,679950.0,835428.571429,697000.0,761857.142857,2.294286e+06
1,0,0,None,기타 미용업,2000.000000,9,0,871571.428571,1005000.0,747000.0,10907.070352,11031.902535,10895.011409,761025.0,804979.761905,679950.0,835428.571429,697000.0,761857.142857,2.294286e+06
2,0,0,None,기타 미용업,6428.571429,9,0,871571.428571,1005000.0,747000.0,10907.070352,11031.902535,10895.011409,761025.0,804979.761905,679950.0,835428.571429,697000.0,761857.142857,2.294286e+06
3,0,0,None,기타 미용업,7142.857143,9,0,871571.428571,1005000.0,747000.0,10907.070352,11031.902535,10895.011409,761025.0,804979.761905,679950.0,835428.571429,697000.0,761857.142857,2.294286e+06
4,0,0,None,기타 미용업,1857.142857,9,0,871571.428571,1005000.0,747000.0,10907.070352,11031.902535,10895.011409,761025.0,804979.761905,679950.0,835428.571429,697000.0,761857.142857,2.294286e+06


### 3.5 Split the dataset 

In [38]:
column_drops=['store_id','region','type_of_business','t','value_next_1','value_next_2','value_next_3','installment_term']
df.drop(columns=column_drops,inplace=True)
df.head()

,amount,is_installment,mean_previous_1,mean_previous_2,mean_previous_3,region_previous_1,region_previous_2,region_previous_3,business_previous-1,business_previous-2,business_previous-3,y
0,2000.000000,0,871571.428571,1005000.0,747000.0,10907.070352,11031.902535,10895.011409,761025.0,804979.761905,679950.0,2.294286e+06
1,2000.000000,0,871571.428571,1005000.0,747000.0,10907.070352,11031.902535,10895.011409,761025.0,804979.761905,679950.0,2.294286e+06
2,6428.571429,0,871571.428571,1005000.0,747000.0,10907.070352,11031.902535,10895.011409,761025.0,804979.761905,679950.0,2.294286e+06
3,7142.857143,0,871571.428571,1005000.0,747000.0,10907.070352,11031.902535,10895.011409,761025.0,804979.761905,679950.0,2.294286e+06
4,1857.142857,0,871571.428571,1005000.0,747000.0,10907.070352,11031.902535,10895.011409,761025.0,804979.761905,679950.0,2.294286e+06


In [39]:
X,y=df.drop(columns='y'),df['y']

In [40]:
##split X,y into the train and test set
from sklearn.model_selection import train_test_split


train_X,test_X,train_y,test_y=train_test_split(X,y,test_size=.3)

In [41]:
train_y.describe()

count    3.884087e+06
mean     4.142256e+06
std      5.644883e+06
min     -1.747857e+05
25%      1.655650e+06
50%      2.895786e+06
75%      4.916543e+06
max      1.727659e+08
Name: y, dtype: float64

### 4. Preprocessing Data

### 4.1 Removing Outliers

In [42]:
def IQR(val):
    Q1=np.quantile(val,0.25)
    Q3=np.quantile(val,0.75)
    IQR=Q3-Q1
    condition=(Q3+1.5*IQR>val)&(Q1-1.5*IQR<val)
    return condition

y_condtion=IQR(train_y)

In [43]:
train_y=train_y[y_condtion]
train_X=train_X[y_condtion]

### 4.2 Meaure Skewness 

In [44]:
columns_drop=['is_installment','amount']

In [45]:
train_X.drop(columns=columns_drop).skew()

mean_previous_1        2.209181
mean_previous_2        1.921953
mean_previous_3        1.961051
region_previous_1      3.976343
region_previous_2      3.980571
region_previous_3      4.002032
business_previous-1    2.226317
business_previous-2    2.736313
business_previous-3    2.788865
dtype: float64

In [46]:
biased_colums=train_X.drop(columns=columns_drop).columns
train_X[biased_colums]=train_X[biased_colums]-train_X[biased_colums].min()+1
train_X[biased_colums]=np.sqrt(train_X[biased_colums])

In [47]:
train_X[biased_colums].skew()

mean_previous_1        0.665061
mean_previous_2        0.598643
mean_previous_3        0.595096
region_previous_1      1.797199
region_previous_2      1.811441
region_previous_3      2.032658
business_previous-1   -0.070475
business_previous-2   -0.064019
business_previous-3   -0.035775
dtype: float64

### 4.3 Feature Scaling 

If there is a great difference in range, the most significant number(i.e., the feature with the larger unit) tend to play a more
deicisive role while training the model. If the large-unit feature gains a less practical importance, our training model will generate a false result. To aovid this, we need to implement feature scaling. 

In [48]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler().fit(train_X)
s_train_X=scaler.transform(train_X)
s_test_X=scaler.transform(test_X)

train_X=pd.DataFrame(s_train_X,columns=train_X.columns)
test_X=pd.DataFrame(s_test_X,columns=test_X.columns)

In [49]:
#remove variables for memory use
del s_train_X,s_test_X

### 5. Modeling

In [52]:
from sklearn.model_selection import ParameterGrid
from sklearn.neighbors import KNeighborsRegressor as KNN
from sklearn.ensemble import RandomForestRegressor as RFR
from lightgbm import LGBMRegressor as LGB
from sklearn.feature_selection import *
from sklearn.metrics import mean_absolute_error as MAE

In [54]:

param_grid = dict() 

param_grid_for_knn = ParameterGrid({"n_neighbors": [1, 3, 5, 7],
                           "metric":['euclidean', 'cosine']})

param_grid_for_RFR = ParameterGrid({"max_depth": [1, 2, 3, 4],
                           "n_estimators":[100, 200],
                                   "max_samples":[0.5, 0.6, 0.7, None]}) # 특징 대비 샘플이 많아서 붓스트랩 비율 (max_samples)을 설정 

param_grid_for_LGB = ParameterGrid({"max_depth": [1, 2, 3, 4],
                                   "n_estimators":[100, 200],
                            "learning_rate": [0.05, 0.1, 0.15]})


param_grid[KNN] = param_grid_for_knn
param_grid[RFR] = param_grid_for_RFR
param_grid[LGB] = param_grid_for_LGB

In [ ]:
# 출력을 위한 max_iter_num 계산
max_iter_num = 0
for k in range(10, 2, -1):
    for M in param_grid.keys():
        for P in param_grid[M]:
            max_iter_num += 1
           


best_score = 9999999999
iteration_num = 0
for k in range(10, 2, -1): # 메모리 부담 해소를 위해, 1씩 감소시킴
    selector = SelectKBest(f_regression, k = k).fit(Train_X, Train_Y)
    selected_features = Train_X.columns[selector.get_support()]

    Train_X = Train_X[selected_features]
    Test_X = Test_X[selected_features]
    
    for M in param_grid.keys():
        for P in param_grid[M]:
            # LightGBM에서 DataFrame이 잘 처리되지 않는 것을 방지하기 위해 .values를 사용
            model = M(**P).fit(Train_X.values, Train_Y.values)
            pred_Y = model.predict(Test_X.values)
            score = MAE(Test_Y.values, pred_Y)
            
            if score < best_score:
                best_score = score
                best_model = M
                best_paramter = P
                best_features = selected_features    
                
            iteration_num += 1
            print("iter_num:{}/{}, score: {}, best_score: {}".format(iteration_num, max_iter_num, round(score, 2), round(best_score, 2)))

In [ ]:
def pipeline(X):
    X[biased_variables] = X[biased_variables] - X[biased_variables].min() + 1
    X[biased_variables] = np.sqrt(X[biased_variables])        
    X = pd.DataFrame(scaler.transform(X), columns = X.columns)
    X = X[best_features]
    return X
    
model = best_model(**best_paramter).fit(pipeline(X).values, Y)